In [5]:
import pandas as pd
import numpy as np
import re

In [6]:
pd.set_option("display.max_colwidth", 510)

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV

In [8]:
from sklearn.metrics import roc_auc_score, roc_curve
from nltk.stem.snowball import EnglishStemmer

_______________________________

# Cocktail recommender:  
User inputs a description of what kind of cocktail she/he likes and the algorithme returns suggestions based on KNN.

## Create table to tokenize

CREATE A MATRIX OF WORDS CONTAINING BOTH 'Ingredients' AND 'Description' WORDS !!

In [141]:
df_ctl_info = pd.read_csv('./CSVs/cocktail_info.csv', sep=',')
ctl_name = pd.read_csv('./CSVs/ctl_name_id.csv', sep=',', index_col=[0])
ctl_ingredient_words = pd.read_csv('./CSVs/ingredient_table.csv', sep=',',index_col=[0])
ctl_ing_id = pd.read_csv('./CSVs/cocktail_ingredient_table.csv', sep=',')

In [69]:
ctl_ing_id.drop(columns='typing.Literal[<no_default>]', inplace=True)

In [70]:
ctl_ing_id.head()

,cocktail_id,ingredient_id,quantity
0,0,461,1 oz
1,0,265,.5 oz
2,0,263,.25 oz
3,0,257,1 oz
4,0,523,1 tsp


In [71]:
ctl_name.head()

,cocktail_id,name
0,0,American Beauty Cocktail
1,1,Azzuro
2,2,Apple Fairy
3,3,Black Rose Bacardi
4,4,Blue Latvian


In [81]:
df_ctl_info

,Name,Description,Recipe,Ingredients,Nutrition Facts,Video Link
0,American Beauty Cocktail,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Woops... We couldn't retrieve the exact recipe... It's trial & error time! Just a little more fun before enjoying a nice drink!,| 1 oz: Brandy | .5 oz: Dry Vermouth | .25 oz: White Creme De Menthe | 1 oz: Orange Juice | 1 tsp: Grenadine Syrup | 1 oz: Red Port,Calories: 185kcal | Carbohydrates: 14g | Protein: 0.3g | Fat: 0.1g | Saturated Fat: 0.01g | Polyunsaturated Fat: 0.02g | Monounsaturated Fat: 0.01g | Sodium: 6mg | Potassium: 89mg | Fiber: 0.1g | Sugar: 10g | Vitamin A: 50IU | Vitamin C: 20.6mg,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
1,Azzuro,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Shake and strain into an ice-filled collins glass. and garnish with fruit.,| 1 oz: Exotic-Fruit Liqueur | 1 oz: Blue Curacao Liqueur | 1 oz: Banana Liqueur | 2 oz: Passion Fruit Juice | 2 oz: Apple Juice | 1 tsp: Egg,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
2,Apple Fairy,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",For this recipe. make some Juice - Apple Juice ice cubes before you start. Then add 3 Juice - Apple Juice cubes to a cocktail glass. Add absinthe and apple Vodka. and fill with cider.,| 2/3 oz: Herbal Liqueur | 2/3 oz: Apple Vodka | 3 oz: Apple Soda,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
3,Black Rose Bacardi,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Shake or stir. pour it into a coctail glass. add some crushed ice and serve.,|: Ice | 1 oz: White Rum | 2 oz: Cola | 2 oz: Lime Juice,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
4,Blue Latvian,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Pour the Jack Daniel`s whiskey into a mixing glass. Add blue raspberry juice until the mixture is more blue than brown. Finish with lemon-lime soda until the mixture is fizzing. Serve in a collins glass.,| 1 oz: Bourbon Whiskey |: Raspberry Juice |: Lemon-Lime Soda,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
...,...,...,...,...,...,...
5111,Brown Hen,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Fill glass with ice. then Vodka. then bitters. Stir.,| 2 oz: Vodka | 1 tsp tsp: Bitters,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
5112,Bosom Caresser,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",In a mixing glass half-filled with crushed ice. combine all of the ingredients. Stir well. Strain into a cocktail glass.,|: Crushed Ice | 1.5 oz: Brandy | 1 oz: Madeira | .5 oz: Triple Sec,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
5113,Brandy Water Press,"Unfortunately, we h

In [142]:
df_ctl_info.head()

,Name,Description,Recipe,Ingredients,Nutrition Facts,Video Link
0,American Beauty Cocktail,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Woops... We couldn't retrieve the exact recipe... It's trial & error time! Just a little more fun before enjoying a nice drink!,| 1 oz: Brandy | .5 oz: Dry Vermouth | .25 oz: White Creme De Menthe | 1 oz: Orange Juice | 1 tsp: Grenadine Syrup | 1 oz: Red Port,Calories: 185kcal | Carbohydrates: 14g | Protein: 0.3g | Fat: 0.1g | Saturated Fat: 0.01g | Polyunsaturated Fat: 0.02g | Monounsaturated Fat: 0.01g | Sodium: 6mg | Potassium: 89mg | Fiber: 0.1g | Sugar: 10g | Vitamin A: 50IU | Vitamin C: 20.6mg,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
1,Azzuro,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Shake and strain into an ice-filled collins glass. and garnish with fruit.,| 1 oz: Exotic-Fruit Liqueur | 1 oz: Blue Curacao Liqueur | 1 oz: Banana Liqueur | 2 oz: Passion Fruit Juice | 2 oz: Apple Juice | 1 tsp: Egg,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
2,Apple Fairy,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",For this recipe. make some Juice - Apple Juice ice cubes before you start. Then add 3 Juice - Apple Juice cubes to a cocktail glass. Add absinthe and apple Vodka. and fill with cider.,| 2/3 oz: Herbal Liqueur | 2/3 oz: Apple Vodka | 3 oz: Apple Soda,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
3,Black Rose Bacardi,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Shake or stir. pour it into a coctail glass. add some crushed ice and serve.,|: Ice | 1 oz: White Rum | 2 oz: Cola | 2 oz: Lime Juice,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
4,Blue Latvian,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Pour the Jack Daniel`s whiskey into a mixing glass. Add blue raspberry juice until the mixture is more blue than brown. Finish with lemon-lime soda until the mixture is fizzing. Serve in a collins glass.,| 1 oz: Bourbon Whiskey |: Raspberry Juice |: Lemon-Lime Soda,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!


In [78]:
df_ctl_dropnandescription = df_ctl_info[df_ctl_info.Description != "Unfortunately, we have no description for this drink... You'll have to describe it yourself!"]

In [79]:
len(df_ctl_dropnandescription)

371

In [174]:
df_ctl_dropnanrecipe = df_ctl_info[df_ctl_info.Recipe != "Woops... We couldn't retrieve the exact recipe... It's trial & error time! Just a little more fun before enjoying a nice drink!"]

In [175]:
len(df_ctl_dropnanrecipe)

5102

In [176]:
df_ctl_dropnanrecipe.head(2)

,Name,Description,Recipe,Ingredients,Nutrition Facts,Video Link
1,Azzuro,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Shake and strain into an ice-filled collins glass. and garnish with fruit.,| 1 oz: Exotic-Fruit Liqueur | 1 oz: Blue Curacao Liqueur | 1 oz: Banana Liqueur | 2 oz: Passion Fruit Juice | 2 oz: Apple Juice | 1 tsp: Egg,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
2,Apple Fairy,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",For this recipe. make some Juice - Apple Juice ice cubes before you start. Then add 3 Juice - Apple Juice cubes to a cocktail glass. Add absinthe and apple Vodka. and fill with cider.,| 2/3 oz: Herbal Liqueur | 2/3 oz: Apple Vodka | 3 oz: Apple Soda,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!


In [178]:
df_ctl_dropnanrecipe.iloc[:,1] # = 'Description'
df_ctl_dropnanrecipe.iloc[0,2] # = 'Recipe'
df_ctl_dropnanrecipe.iloc[0,3] # = 'Ingredients'

' | 1 oz: Exotic-Fruit Liqueur | 1 oz: Blue Curacao Liqueur | 1 oz: Banana Liqueur | 2 oz: Passion Fruit Juice | 2 oz: Apple Juice | 1 tsp: Egg'

In [184]:
def use_regex(input_text):
    pattern = re.sub(r'\b\d+(\.\d+)?\s*\w+\b|[^\w\s]', '', string = input_text).lower()
    return pattern

In [185]:
df_ctl_dropnanrecipe.iloc[:,1] = df_ctl_dropnanrecipe.iloc[:,1].apply(lambda x: use_regex(x))

In [186]:
df_ctl_dropnanrecipe.iloc[:,2] = df_ctl_dropnanrecipe.iloc[:,2].apply(lambda x: use_regex(x))

In [187]:
df_ctl_dropnanrecipe.iloc[:,3] = df_ctl_dropnanrecipe.iloc[:,3].apply(lambda x: use_regex(x))

In [230]:
len(df_ctl_dropnanrecipe)

5102

replace "unfortunately we have no description for this drink youll have to describe it yourself"  
with ""  (empty string)

In [ ]:
df_ctl_dropnanrecipe = df_ctl_info[df_ctl_info.Recipe == "unfortunately we have no description for this drink youll have to describe it yourself"]

In [237]:
df_ctl_dropnanrecipe.iloc[:,1] = df_ctl_dropnanrecipe.iloc[:,1].replace("unfortunately we have no description for this drink youll have to describe it yourself", "")

In [280]:
df_ctl_dropnanrecipe.head()

,Name,Description,Recipe,Ingredients,Nutrition Facts,Video Link
1,Azzuro,,shake and strain into an icefilled collins glass and garnish with fruit,exoticfruit liqueur blue curacao liqueur banana liqueur passion fruit juice apple juice egg,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
2,Apple Fairy,,for this recipe make some juice apple juice ice cubes before you start then add apple juice cubes to a cocktail glass add absinthe and apple vodka and fill with cider,liqueur vodka apple soda,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
3,Black Rose Bacardi,,shake or stir pour it into a coctail glass add some crushed ice and serve,ice white rum cola lime juice,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
4,Blue Latvian,,pour the jack daniels whiskey into a mixing glass add blue raspberry juice until the mixture is more blue than brown finish with lemonlime soda until the mixture is fizzing serve in a collins glass,bourbon whiskey raspberry juice lemonlime soda,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
5,Boston Cooler,,stir lemon juice sugar and of club soda together in a collins glass fill the glass with cracked ice add rum top with club soda and stir garnish with a spiral of orange or lemon peel and serve,white rum lemon juice superfine sugar soda,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!


Create matrix/tokenized dataframes :  
* 'Description' matrix  
* 'Recipe' matrix
* 'Ingredients' matrix  

and later, merge them on cocktail name.

In [239]:
vectorizer = CountVectorizer(strip_accents='ascii', stop_words='english')

In [255]:
index = df_ctl_dropnanrecipe['Name']

Description matrix

In [240]:
desc_words = df_ctl_dropnanrecipe.iloc[:,1].apply(lambda x: ' '.join(x.split())).tolist()

In [246]:
desc_vectors = vectorizer.fit_transform(desc_words).todense()

In [247]:
len(desc_vectors)

5102

In [248]:
desc_vocabulary = vectorizer.get_feature_names_out()

In [256]:
desc_matrix = pd.DataFrame(desc_vectors, columns=desc_vocabulary, index=index)

In [257]:
desc_matrix

,abc,absinthe,absinthesazerac,absolutely,absorbs,accent,acceptance,accepted,accessible,accompanied,...,youd,youll,young,youre,youtube,youve,yummy,yvette,zesty,zombie
Name,,,,,,,,,,,,,,,,,,,,,
Azzuro,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Apple Fairy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Black Rose Bacardi,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Blue Latvian,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Boston Cooler,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Brown Hen,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Bosom Caresser,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Brandy Water Press,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Recipe matrix

In [263]:
recipe_words = df_ctl_dropnanrecipe.iloc[:,2].apply(lambda x: ' '.join(x.split())).tolist()

In [264]:
recipe_vectors = vectorizer.fit_transform(recipe_words).todense()

In [265]:
len(recipe_vectors)

5102

In [281]:
recipe_vocabulary = vectorizer.get_feature_names_out()

In [282]:
recipe_matrix = pd.DataFrame(recipe_vectors, columns=recipe_vocabulary, index=index)

In [283]:
recipe_matrix = recipe_matrix.drop(columns=['aaa','aaaaaaaaasaagermeister'])

In [284]:
recipe_matrix

,ab,able,absente,absinthe,absolut,absolute,absorb,absorbed,absorbs,acceptable,...,yum,zambuca,zealand,zest,zigzag,zima,zinggarnish,zombie,zubrowka,zwack
Name,,,,,,,,,,,,,,,,,,,,,
Azzuro,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Apple Fairy,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Black Rose Bacardi,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Blue Latvian,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Boston Cooler,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Brown Hen,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Bosom Caresser,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Brandy Water Press,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Ingredient matrix

In [285]:
ing_words = df_ctl_dropnanrecipe.iloc[:,3].apply(lambda x: ' '.join(x.split())).tolist()

In [286]:
ing_vectors = vectorizer.fit_transform(ing_words).todense()

In [287]:
len(ing_vectors)

5102

In [288]:
ing_vocabulary = vectorizer.get_feature_names_out()

In [289]:
ing_matrix = pd.DataFrame(ing_vectors, columns=ing_vocabulary, index=index)

In [290]:
ing_matrix

,12,14,18,34,absinthe,advocaat,agave,aged,alcohol,alcoholic,...,withcoffee,withcream,withlemonade,woodruff,worcestershire,wormwood,yeast,yellow,yoghurt,yolk
Name,,,,,,,,,,,,,,,,,,,,,
Azzuro,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Apple Fairy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Black Rose Bacardi,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Blue Latvian,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Boston Cooler,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Brown Hen,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Bosom Caresser,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Brandy Water Press,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Merge the matrixes on 'Name'

In [ ]:
desc_matrix
recipe_matrix
ing_matrix

df_temp

In [291]:
df_temp_merge = pd.merge(desc_matrix,recipe_matrix, on='Name')

In [292]:
df_merged = pd.merge(df_temp_merge,ing_matrix,on='Name' )

In [293]:
df_merged

,abc,absinthe_x,absinthesazerac,absolutely,absorbs_x,accent,acceptance,accepted,accessible,accompanied,...,withcoffee,withcream,withlemonade,woodruff_y,worcestershire_y,wormwood_y,yeast_y,yellow_y,yoghurt_y,yolk_y
Name,,,,,,,,,,,,,,,,,,,,,
Azzuro,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Azzuro,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Azzuro,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Azzuro,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Azzuro,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Brown Hen,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Bosom Caresser,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Brandy Water Press,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [295]:
df_merged.drop_duplicates(inplace=True)

In [296]:
df_merged

,abc,absinthe_x,absinthesazerac,absolutely,absorbs_x,accent,acceptance,accepted,accessible,accompanied,...,withcoffee,withcream,withlemonade,woodruff_y,worcestershire_y,wormwood_y,yeast_y,yellow_y,yoghurt_y,yolk_y
Name,,,,,,,,,,,,,,,,,,,,,
Azzuro,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Apple Fairy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Black Rose Bacardi,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Blue Latvian,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Boston Cooler,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Brown Hen,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Bosom Caresser,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Brandy Water Press,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Clean data for tokenization

In [225]:
vectors = vectorizer.fit_transform(word_list).todense()

In [226]:
len(vectors)

15306

In [227]:
vocabulary = vectorizer.get_feature_names_out()

In [221]:
len(vocabulary)

4677

In [228]:
index = df_ctl_dropnanrecipe['Name']

In [229]:
len(index)

5102

In [209]:
token_matrix = pd.DataFrame(vectors, columns=vocabulary)

In [210]:
token_matrix.head(2)

,ab,abc,able,absente,absinthe,absinthesazerac,absolut,absolute,absolutely,absorb,...,zest,zesty,zigzag,zima,zinggarnish,zombie,zubrowka,zwack,ãââ,ãââãƒæãâåãƒâšãâgermeister
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
pd.set_option('display.max_columns', 507)
token_matrix

,absinthe,advocaat,agave,aged,alcohol,alcoholic,ale,allspice,almond,amaro,amarula,amber,american,amp,angelica,angostura,anis,anise,aperol,apple,apples,apricot,aquavit,arak,bacardi,baileys,banana,bananas,bananastrawberry,bananes,bannanas,bar,base,basil,bbq,beans,bears,beaten,beef,beer,beers,benedictine,berries,berry,birch,bison,bitter,bitters,black,blackberries,blackberry,blackcurrant,blackcurrants,blended,blood,bloodorange,bloody,blue,blueberries,blueberry,boiling,bois,bollinger,bottle,bouillon,bourbon,brandy,bread,breezer,brewed,bristow,brown,bull,burgundy,butter,butterfly,butterscotch,cacao,cachaca,cactus,cake,campari,canadian,candied,candy,cantaloupe,cappuccino,caramel,caraway,cardamom,caribbean,carrot,cassis,caster,cayenne,celery,chai,champagne,chardonnay,chartreuse,cheese,cherries,cherry,chicken,chili,chilli,chips,chocchip,chocolate,chopped,cider,cinnamon,citrus,clam,clamato,clementine,cloudberry,cloves,club,coast,cocktail,cocoa,coconut,coffee,cognac,cola,colada,cold,collins,colour,colouring,comfort,concentrate,condensed,congnac,conserve,cookies,cool,cooler,cordial,coriander,corn,cornstarch,corona,coronita,cottage,cracked,crackers,cranberries,cranberry,cranberryapple,cranberrygrape,cranberryraspberry,cream,crem,creme,crushed,cube,cubes,cucumber,cucumbers,cup,cups,curacao,daiquiri,dark,dashes,demerara,des,didnt,dill,distilled,divided,double,draft,dragonberry,drink,dry,dust,eclair,edible,egg,eggnog,elderflower,energy,evaporated,exoticfruit,extract,eye,fennel,fills,finely,fireball,float,flowers,food,forbidden,fraise,frambois,fresh,freshly,frozen,fruit,garnish,gatorade,gelatin,gilbey,gin,giner,ginger,gingerbread,glow,glycerine,gold,gomme,grain,grape,grapefruit,grapes,grass,grated,green,grenadine,ground,guava,guavaberry,guinness,gummi,halfandhalf,happened,hard,harmonie,hazelnut,hazelnuts,heavy,herbal,honey,horseradish,hot,hpnotiq,ice,iced,icewine,inch,infused,irish,italian,jager,jalapeno,jamaican,jello,jelly,juice,kiwi,kiwistrawberry,koolaid,kryptonite,kumquat,lace,lamb,larger,lavender,layer,leaves,lemon,lemonade,lemonlime,lemons,licorice,light,lime,liqueur,llemonlime,lots,luxardo,lychee,lychees,mace,madeira,malt,mandarin,mango,mangos,maple,maraschino,margarita,marshmallows,mary,mashed,melon,men,menthe,merlot,mexican,mezcal,milk,mineral,mint,mix,mixed,molasses,moscato,mure,muscatel,mustard,nectar,noodle,nutella,nutmeg,nuts,ojen,olive,olives,onion,op,optional,orange,orangebanana,oranges,oregano,oreo,organic,orgeat,original,ounce,oz,papaya,passion,passionfruit,pea,peach,peaches,peachmango,peachs,peanut,peanuts,pear,pearl,pears,pecan,peel,pepper,peppermint,peppers,pernod,peychauds,pickle,pimms,pina,pinch,pineapple,pineapples,pineorangebanana,pink,pisco,pitch,plum,plums,pomegranate,port,powder,powdered,preferred,premix,prosecco,prune,pumpkin,punch,puree,rainbow,raisins,raspberries,raspberry,recipe,recommended,red,redcurrant,rice,rind,rinse,rock,root,rosemary,ruby,rum,rye,sage,salt,sambuca,sarsaparilla,sauce,schanpps,schnapps,scotch,seasonal,sec,serve,serving,shavings,sherbert,sherbet,sherry,shots,shredded,silver,simple,slice,sliced,slices,sloe,smirnoff,soda,soft,soup,sour,sours,southern,soy,sparkling,spice,spiced,spicy,splash,spoon,sprite,squash,squeezed,stalk,starfruits,steak,stick,stout,straw,strawberries,strawberry,strawberrybanana,strawberrykiwi,strega,strips,sugar,superfine,sweet,syrup,tabasco,tajin,tamarillo,tangerine,tea,tennessee,tequila,toffee,tomato,tonic,topping,triple,tropical,tropicalfruit,tropico,try,tsp,tsps,tuaca,twist,unflavored,unflavoured,vanilla,vegetable,vermouth,version,vinegar,viniq,violette,vitamin,voda,vodak,vodka,walnut,water,watermelon,watermeloncherry,wedge,wedges,welch,west,wheel,wheels,whipped,whipping,whiskey,whisky,white,wildberry,wilderberry,wine,withchampagne,withcoffee,withcream,withlemonade,woodruff,woops,worcestershire,work,wormwood,yeast,yellow,yoghurt,yolk
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

## Preprocess data

In [41]:
X = token_matrix.reset_index()
X.head()

,Name,absinthe,advocaat,agave,aged,alcohol,alcoholic,ale,allspice,almond,amaro,amarula,amber,american,amp,angelica,angostura,anis,anise,aperol,apple,apples,apricot,aquavit,arak,bacardi,baileys,banana,bananas,bananastrawberry,bananes,bannanas,bar,base,basil,bbq,beans,bears,beaten,beef,beer,beers,benedictine,berries,berry,birch,bison,bitter,bitters,black,blackberries,blackberry,blackcurrant,blackcurrants,blended,blood,bloodorange,bloody,blue,blueberries,blueberry,boiling,bois,bollinger,bottle,bouillon,bourbon,brandy,bread,breezer,brewed,bristow,brown,bull,burgundy,butter,butterfly,butterscotch,cacao,cachaca,cactus,cake,campari,canadian,candied,candy,cantaloupe,cappuccino,caramel,caraway,cardamom,caribbean,carrot,cassis,caster,cayenne,celery,chai,champagne,chardonnay,chartreuse,cheese,cherries,cherry,chicken,chili,chilli,chips,chocchip,chocolate,chopped,cider,cinnamon,citrus,clam,clamato,clementine,cloudberry,cloves,club,coast,cocktail,cocoa,coconut,coffee,cognac,cola,colada,cold,collins,colour,colouring,comfort,concentrate,condensed,congnac,conserve,cookies,cool,cooler,cordial,coriander,corn,cornstarch,corona,coronita,cottage,cracked,crackers,cranberries,cranberry,cranberryapple,cranberrygrape,cranberryraspberry,cream,crem,creme,crushed,cube,cubes,cucumber,cucumbers,cup,cups,curacao,daiquiri,dark,dashes,demerara,des,didnt,dill,distilled,divided,double,draft,dragonberry,drink,dry,dust,eclair,edible,egg,eggnog,elderflower,energy,evaporated,exoticfruit,extract,eye,fennel,fills,finely,fireball,float,flowers,food,forbidden,fraise,frambois,fresh,freshly,frozen,fruit,garnish,gatorade,gelatin,gilbey,gin,giner,ginger,gingerbread,glow,glycerine,gold,gomme,grain,grape,grapefruit,grapes,grass,grated,green,grenadine,ground,guava,guavaberry,guinness,gummi,halfandhalf,happened,hard,harmonie,hazelnut,hazelnuts,heavy,herbal,honey,horseradish,hot,hpnotiq,ice,iced,icewine,inch,infused,irish,italian,jager,jalapeno,jamaican,jello,jelly,...,kiwistrawberry,koolaid,kryptonite,kumquat,lace,lamb,larger,lavender,layer,leaves,lemon,lemonade,lemonlime,lemons,licorice,light,lime,liqueur,llemonlime,lots,luxardo,lychee,lychees,mace,madeira,malt,mandarin,mango,mangos,maple,maraschino,margarita,marshmallows,mary,mashed,melon,men,menthe,merlot,mexican,mezcal,milk,mineral,mint,mix,mixed,molasses,moscato,mure,muscatel,mustard,nectar,noodle,nutella,nutmeg,nuts,ojen,olive,olives,onion,op,optional,orange,orangebanana,oranges,oregano,oreo,organic,orgeat,original,ounce,oz,papaya,passion,passionfruit,pea,peach,peaches,peachmango,peachs,peanut,peanuts,pear,pearl,pears,pecan,peel,pepper,peppermint,peppers,pernod,peychauds,pickle,pimms,pina,pinch,pineapple,pineapples,pineorangebanana,pink,pisco,pitch,plum,plums,pomegranate,port,powder,powdered,preferred,premix,prosecco,prune,pumpkin,punch,puree,rainbow,raisins,raspberries,raspberry,recipe,recommended,red,redcurrant,rice,rind,rinse,rock,root,rosemary,ruby,rum,rye,sage,salt,sambuca,sarsaparilla,sauce,schanpps,schnapps,scotch,seasonal,sec,serve,serving,shavings,sherbert,sherbet,sherry,shots,shredded,silver,simple,slice,sliced,slices,sloe,smirnoff,soda,soft,soup,sour,sours,southern,soy,sparkling,spice,spiced,spicy,splash,spoon,sprite,squash,squeezed,stalk,starfruits,steak,stick,stout,straw,strawberries,strawberry,strawberrybanana,strawberrykiwi,strega,strips,sugar,superfine,sweet,syrup,tabasco,tajin,tamarillo,tangerine,tea,tennessee,tequila,toffee,tomato,tonic,topping,triple,tropical,tropicalfruit,tropico,try,tsp,tsps,tuaca,twist,unflavored,unflavoured,vanilla,vegetable,vermouth,version,vinegar,viniq,violette,vitamin,voda,vodak,vodka,walnut,water,watermelon,watermeloncherry,wedge,wedges,welch,west,wheel,wheels,whipped,whipping,whiskey,whisky,white,wildberry,wilderberry,wine,withchampagne,withcoffee,withcream,withlemonade,woodruff,woops,worcestershire,work,wormwood,yeast,yellow,yoghurt,yolk
0,American Beauty Cocktail,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [42]:
X = X.drop('Name', axis=1)
X.head()

,absinthe,advocaat,agave,aged,alcohol,alcoholic,ale,allspice,almond,amaro,amarula,amber,american,amp,angelica,angostura,anis,anise,aperol,apple,apples,apricot,aquavit,arak,bacardi,baileys,banana,bananas,bananastrawberry,bananes,bannanas,bar,base,basil,bbq,beans,bears,beaten,beef,beer,beers,benedictine,berries,berry,birch,bison,bitter,bitters,black,blackberries,blackberry,blackcurrant,blackcurrants,blended,blood,bloodorange,bloody,blue,blueberries,blueberry,boiling,bois,bollinger,bottle,bouillon,bourbon,brandy,bread,breezer,brewed,bristow,brown,bull,burgundy,butter,butterfly,butterscotch,cacao,cachaca,cactus,cake,campari,canadian,candied,candy,cantaloupe,cappuccino,caramel,caraway,cardamom,caribbean,carrot,cassis,caster,cayenne,celery,chai,champagne,chardonnay,chartreuse,cheese,cherries,cherry,chicken,chili,chilli,chips,chocchip,chocolate,chopped,cider,cinnamon,citrus,clam,clamato,clementine,cloudberry,cloves,club,coast,cocktail,cocoa,coconut,coffee,cognac,cola,colada,cold,collins,colour,colouring,comfort,concentrate,condensed,congnac,conserve,cookies,cool,cooler,cordial,coriander,corn,cornstarch,corona,coronita,cottage,cracked,crackers,cranberries,cranberry,cranberryapple,cranberrygrape,cranberryraspberry,cream,crem,creme,crushed,cube,cubes,cucumber,cucumbers,cup,cups,curacao,daiquiri,dark,dashes,demerara,des,didnt,dill,distilled,divided,double,draft,dragonberry,drink,dry,dust,eclair,edible,egg,eggnog,elderflower,energy,evaporated,exoticfruit,extract,eye,fennel,fills,finely,fireball,float,flowers,food,forbidden,fraise,frambois,fresh,freshly,frozen,fruit,garnish,gatorade,gelatin,gilbey,gin,giner,ginger,gingerbread,glow,glycerine,gold,gomme,grain,grape,grapefruit,grapes,grass,grated,green,grenadine,ground,guava,guavaberry,guinness,gummi,halfandhalf,happened,hard,harmonie,hazelnut,hazelnuts,heavy,herbal,honey,horseradish,hot,hpnotiq,ice,iced,icewine,inch,infused,irish,italian,jager,jalapeno,jamaican,jello,jelly,juice,kiwi,kiwistrawberry,koolaid,kryptonite,kumquat,lace,lamb,larger,lavender,layer,leaves,lemon,lemonade,lemonlime,lemons,licorice,light,lime,liqueur,llemonlime,lots,luxardo,lychee,lychees,mace,madeira,malt,mandarin,mango,mangos,maple,maraschino,margarita,marshmallows,mary,mashed,melon,men,menthe,merlot,mexican,mezcal,milk,mineral,mint,mix,mixed,molasses,moscato,mure,muscatel,mustard,nectar,noodle,nutella,nutmeg,nuts,ojen,olive,olives,onion,op,optional,orange,orangebanana,oranges,oregano,oreo,organic,orgeat,original,ounce,oz,papaya,passion,passionfruit,pea,peach,peaches,peachmango,peachs,peanut,peanuts,pear,pearl,pears,pecan,peel,pepper,peppermint,peppers,pernod,peychauds,pickle,pimms,pina,pinch,pineapple,pineapples,pineorangebanana,pink,pisco,pitch,plum,plums,pomegranate,port,powder,powdered,preferred,premix,prosecco,prune,pumpkin,punch,puree,rainbow,raisins,raspberries,raspberry,recipe,recommended,red,redcurrant,rice,rind,rinse,rock,root,rosemary,ruby,rum,rye,sage,salt,sambuca,sarsaparilla,sauce,schanpps,schnapps,scotch,seasonal,sec,serve,serving,shavings,sherbert,sherbet,sherry,shots,shredded,silver,simple,slice,sliced,slices,sloe,smirnoff,soda,soft,soup,sour,sours,southern,soy,sparkling,spice,spiced,spicy,splash,spoon,sprite,squash,squeezed,stalk,starfruits,steak,stick,stout,straw,strawberries,strawberry,strawberrybanana,strawberrykiwi,strega,strips,sugar,superfine,sweet,syrup,tabasco,tajin,tamarillo,tangerine,tea,tennessee,tequila,toffee,tomato,tonic,topping,triple,tropical,tropicalfruit,tropico,try,tsp,tsps,tuaca,twist,unflavored,unflavoured,vanilla,vegetable,vermouth,version,vinegar,viniq,violette,vitamin,voda,vodak,vodka,walnut,water,watermelon,watermeloncherry,wedge,wedges,welch,west,wheel,wheels,whipped,whipping,whiskey,whisky,white,wildberry,wilderberry,wine,withchampagne,withcoffee,withcream,withlemonade,woodruff,woops,worcestershire,work,wormwood,yeast,yellow,yoghurt,yolk
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,

In [43]:
y = token_matrix.reset_index()
y = y['Name']
y.head()

0    American Beauty Cocktail
1                      Azzuro
2                 Apple Fairy
3          Black Rose Bacardi
4                Blue Latvian
Name: Name, dtype: object

## Split train/test

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Train model

In [93]:
model = NearestNeighbors(n_neighbors=5)

model.fit(X_train.values, y_train.values)

NearestNeighbors()

_______________________________

## User input preprocessing

In [64]:
input_vectorizer = CountVectorizer(stop_words='english')

In [65]:
# vectorizer.fit(X)

CountVectorizer(stop_words='english')

In [94]:
def preprocess_input(input):
    input = input.lower()
    input = re.sub("[0-9]", "",input)
    input = [input]
    input = vectorizer.transform(input)
    return input

In [95]:
user_input = input('Please enter an ingredient')
print(user_input)
input_vector = preprocess_input(user_input)
print(input_vector)

whiskey
  (0, 488)	1


## Finding similar ingredients

In [96]:
distances, indices = model.kneighbors(input_vector, n_neighbors=5)

In [97]:
similar_ingredients = y_train.iloc[indices[0]].values

In [98]:
similar_ingredients

array(['JB Black and Pepsi', 'Blue Bomb', 'Frisky', 'Jack and Coke',
       'Christina Ricci Cocktail'], dtype=object)

In [99]:
recommended_ing = []
print("Recommended ingredients:")
for ingredient in similar_ingredients:
    print(ingredient)

Recommended ingredients:
JB Black and Pepsi
Blue Bomb
Frisky
Jack and Coke
Christina Ricci Cocktail


## Full cocktail info

In [91]:
ctl_info = pd.read_csv('./CSVs/cocktail_info.csv', sep=',')

In [100]:
filtered_df = ctl_info.loc[ctl_info['Name'].isin(similar_ingredients)]
filtered_df

,Name,Description,Recipe,Ingredients,Nutrition Facts,Video Link
583,Blue Bomb,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Pour both ingredients into a beer glass. Drink fast.,| 1.5 - 2 oz: Vodka,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
1516,Christina Ricci Cocktail,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Pour the Scotch whisky into a whiskey sour glass filled with ice. Add the spoonful of coffee. and serve.,| 1 Glass(s): Scotch Whiskey | 1 tbsp: Coffee,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
2171,Frisky,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Mix the Fresca and the whisky and drink fast.,| 1 Part(s): Whiskey | 1 Part(s): Grapefruit Soda,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
2910,JB Black and Pepsi,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Fill a glass with about a 1/5 of ice. then pour Jim Beam Black Label until ice is covered. Fill the rest with Pepsi cola. and serve.,| 2 - 3 oz: Bourbon Whiskey,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
2938,Jack and Coke,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Pour jack daniels into large glass filled with ice. Pour coca-cola into the glass. Stir lightly.,| 2 oz: Bourbon Whiskey | 10 oz: Cola,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!


______________________________

__________________________________

fter fitting the model with the training data using model.fit(X_train, y_train), the next steps would involve:

* Preprocessing the user input:

Transform the user input into a TF-IDF vector using the same TfidfVectorizer that was used for the training data.
Make sure to preprocess the user input in the same way as the training data (e.g., lowercase, remove punctuation, etc.).  

* Finding similar ingredients:

Transform the preprocessed user input into a feature vector using the trained TfidfVectorizer: input_vector = vectorizer.transform([user_input]).  
  
Use the kneighbors() method of the trained model to find the nearest neighbors based on the input vector:  
distances, indices = model.kneighbors(input_vector, n_neighbors=3).  

The indices array will contain the indices of the nearest neighbors in the training data.  

Retrieve the corresponding ingredient names from the training data using these indices: similar_ingredients = y_train[indices].  

* Displaying the results:

Present the similar_ingredients to the user as the recommended ingredients that are likely to appear together with the user inputted ingredient in multiple cocktails.

___________________________

In [ ]:
k = 5
model = NearestNeighbors(n_neighbors=k, metric='hamming')  # 'hamming' for binary data

# Fit the data to the KNN model
knn.fit(X)

# Find the k nearest neighbors for each cocktail
distances, indices = knn.kneighbors(X)

# Iterate over the indices to analyze the neighbors
for i, cocktail_indices in enumerate(indices):
    cocktail_ingredients = []
    for idx in cocktail_indices:
        ingredients = ...  # Get the ingredients corresponding to the index
        cocktail_ingredients.extend(ingredients)